Build (or rebuild) the ``pillars`` package. Run this cell **ONLY WHEN** the local package is out-of-date.

In [ ]:
//sys.process.stringToProcess("make -C .. build").!

Load the library dependencies (see ``load-ivy.sc`` for details).

In [ ]:
val ivy_path = System.getProperty("user.dir") + "/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(ivy_path)))

A example for testing a 2×2 TileBlock with 2 input ports and 1 output port.

In [ ]:
import chisel3.iotesters
import tetriski.pillars.core.{ArchitectureHierarchy, Connect, ConstInfo, HardwareGenerator}
import tetriski.pillars.archlib.TileBlock
import tetriski.pillars.hardware.TopModule
import tetriski.pillars.testers.TopModuleAdresUnitTest

/** A example for testing a 2*2 TileBlock with 2 input ports and 1 output port.
 * "internalNodeinfo_simple.txt" is a pre-generated information TXT
 * when mapping a simple DFG with add and mul.
 */
def exampleAdres(): Unit = {
    val arch = new ArchitectureHierarchy()
    //The order of ports should be same as TopModule
    arch.addOutPorts(Array("output"))
    arch.addInPorts(Array("input_0", "input_1"))

    val dataWidth = 32
    val tile = new TileBlock("tile_0", 2, 2, 2, 1, dataWidth = dataWidth)

    arch.addBlock(tile)

    arch.addConnect(List(List("input_0"), List("tile_0/", "input_0")))
    arch.addConnect(List(List("input_1"), List("tile_0/", "input_1")))
    arch.addConnect(List(List("tile_0/", "out_0"), List("output")))

    arch.init()

    arch.blockMap("tile_0").dumpMRRG(1)

    arch.dumpArchitecture()

    val connectArray = arch.connectArray

    val connect = new Connect(connectArray)

    connect.dumpConnect()

    val hardwareGenerator = new HardwareGenerator(arch, connect)

    // Verilog generation
    chisel3.Driver.execute(Array("--no-check-comb-loops", "-td", "ADRESv0"),
      () => new TopModule(hardwareGenerator.pillarsModuleInfo,
      hardwareGenerator.connectMap, hardwareGenerator.regionList, dataWidth))

    val constInfo = new ConstInfo(1)
    constInfo.addConfig(arch("tile_0")("pe_0_0").getElement("const0").getModuleID(), 0, 4)
    constInfo.addConfig(arch("tile_0")("pe_1_0").getElement("const0").getModuleID(), 0, 5)

    val bitStreams = arch.genConfig("app-mapping-results/hardware-test/internalNodeinfo_simple.txt",
        1, constInfo)

    val bitStream = bitStreams(0)

    // Run tester
    iotesters.Driver.execute(Array("--no-check-comb-loops", "-tgvo", "on", "-tbn", "verilator"),
      () => new TopModule(hardwareGenerator.pillarsModuleInfo,
      hardwareGenerator.connectMap, hardwareGenerator.regionList, dataWidth)) {
        c => new TopModuleAdresUnitTest(c, bitStream)
    }
}

// executing ``exampleAdres()`` will fail due to the dollar ($) sign in Ammonite's temporary naming
tetriski.pillars.examples.HardwareExamples.exampleAdres()